In [1]:
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import adam
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [2]:
def loadTrainData():
    train_data = pd.read_csv('./digit_train.csv')
    x_train = train_data.values[:, 1:]
    y_train = train_data.values[:, 0]
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_train = x_train/255
    y = np.zeros((y_train.shape[0], 10))
    for i in range(y_train.shape[0]):
        y[i][y_train[i]] = 1
    return x_train, y

In [3]:
def loadTestData():
    test_data = pd.read_csv('./digit_test.csv')
    x_test = test_data.values[:]
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_test = x_test/255
    return x_test
    

In [4]:
# 利用keras训练模型
def train_model(x_train, y_train):
    model = Sequential()
    
    model.add(Conv2D(input_shape=(28, 28, 1), kernel_size=(3, 3), filters=32, padding='same', activation='relu'))
    model.add(Conv2D(kernel_size=(3, 3), filters=32, padding='same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    
    ad = keras.optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=ad, metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='acc', patience=2)
    model.fit(x_train, y_train, epochs=50, batch_size=2000, validation_split=0.2, callbacks=[early_stopping])

    model.save('./my_model2.h5')

In [5]:
def predict():
    x_test = loadTestData()
    model_predict = load_model('./my_model2.h5')
    result = model_predict.predict(x_test, batch_size=1000)
    final_result = np.zeros(result.shape[0])
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            if result[i][j] > 0.5:
                final_result[i] = j
    return final_result

In [6]:
def strat_train():
    x_train, y_train = loadTrainData()
    train_model(x_train, y_train)

In [7]:
def save_result(final_result):
    print(final_result.shape[0])
    x = range(1, final_result.shape[0]+1)
    DataFrame = pd.DataFrame({'ImageId': x, 'Label': final_result.astype(int)})
    DataFrame.to_csv('result.csv', index=False, sep=',')
    

In [8]:
strat_train()
final_result = predict()
save_result(final_result)
print('Finished')

Train on 33600 samples, validate on 8400 samples
Epoch 1/50
33600/33600 [==============================] - 34s 1ms/step - loss: 0.9091 - acc: 0.7101 - val_loss: 0.2314 - val_acc: 0.9298
Epoch 2/50
33600/33600 [==============================] - 33s 970us/step - loss: 0.2405 - acc: 0.9291 - val_loss: 0.1101 - val_acc: 0.9662
Epoch 3/50
33600/33600 [==============================] - 36s 1ms/step - loss: 0.1394 - acc: 0.9586 - val_loss: 0.0790 - val_acc: 0.9742
Epoch 4/50
33600/33600 [==============================] - 36s 1ms/step - loss: 0.1020 - acc: 0.9697 - val_loss: 0.0672 - val_acc: 0.9785
Epoch 5/50
33600/33600 [==============================] - 35s 1ms/step - loss: 0.0828 - acc: 0.9748 - val_loss: 0.0555 - val_acc: 0.9815
Epoch 6/50
33600/33600 [==============================] - 33s 977us/step - loss: 0.0688 - acc: 0.9791 - val_loss: 0.0514 - val_acc: 0.9844
Epoch 7/50
33600/33600 [==============================] - 33s 977us/step - loss: 0.0598 - acc: 0.9821 - val_loss: 0.0482 - va